In [4]:
# CREATE POPULATION functions

import pandas as pd
import xml.etree.ElementTree as ET
import random

def get_root(xml_file):
    '''
    Parse through given xml file to obtain the root

    Parameters:
    - xml_file:  the xml file

    Returns:
    
    element: Root of the file
    '''
    return ET.parse(xml_file).getroot()

def generate_cost_data(root):
    '''
    Extract the raw data to create a dicitionary containing the information of all the possible costs from travelling from city i to city j
            
    Parameters:        
    - root: the root of the xml file

    Returns:

    node_cost(dict) : Cities as key, list of all other cities to travel to as value
    '''
    node_cost = {}
    city_id = 0

    for vertex in root.find('graph').findall('vertex'):
        city_connections = {}

        for edge in vertex[0:]:
            to_other_city = int(edge.text)
            cost = float(edge.get('cost'))
            city_connections[to_other_city] = cost

        node_cost[city_id] = city_connections
        city_id += 1

    return node_cost

def initial_population(node_cost, population_size, seed_value):
    '''
    Generate our initial random population of potential routes

    Parameters:

    - node_cost (dict): dictionary containing all city to city information
    - population_size (int): size of the desired population
    - seed_value (int): value for random seed

    Returns:

    Population (list): List of lists of random routes the salesman can take
    
    '''

    random.seed(seed_value)
    population = []
    cities = list(node_cost.keys())

    while len(population) < population_size:
        route = list(random.sample(cities, len(cities)))  
        if route not in population:
            population.append(route)

    return population

def calculate_route_cost(route, node_cost):
    '''
    Calculate the total cost of a route

    Parameters:

    - route (list): list containing a route between cities
    - node_dictionary (dict): dictionary containing all city to city information

    Returns:

    total_cost (float): the total cost of a given route
    '''
    total_cost = 0

    for i in range(len(route) - 1):
        current_city = route[i]
        next_city = route[i + 1]
        total_cost += node_cost[current_city][next_city]

    total_cost += node_cost[route[-1]][route[0]]
    return total_cost

def initialize_population(xml_file, population_size, seed_value):
    '''
    Creates the dataframe, storing population of routes and their associated costs

    Parameters:

    - xml_file (str): file path for raw data
    - population size (int): size of the desired population
    - seed_value (int) - value for the random seed

    Returns:

    - df (DataFrame): dataframe containing column for population, column for the cost
    - node_cost (dictionary): cities as key, list of all other cities to travel to as value

    
    '''
    root = get_root(xml_file)
    node_cost = generate_cost_data(root)
    population = initial_population(node_cost, population_size, seed_value)

    data = []
    for route in population:
        route_cost = calculate_route_cost(route, node_cost)
        data.append({'Population': route, 'Cost': route_cost})

    df = pd.DataFrame(data)
    return df, node_cost

In [5]:
# MUTATIONS functions
# TOURNAMENT
def tournament(df, n):
    '''
    Tournament that takes an n number of routes from the population and produces a single winner (twice)

    Parameters:
    
    - df (pd.DataFrame): DataFrame containing 'Population' and 'Cost' columns
    - n (int): size of the tournament (number of routes taken)

    Returns:
    
    a, b (pd.Series): rows of tournament winners (routes and costs)
    '''
    # Initialize the winners
    a = None
    b = None

    # Repeat the tournament process twice
    for _ in range(2):
        # Randomly select n candidates, winner with minimum cost
        min_cost = min(df['Cost'].sample(n))
        winners = df.loc[df['Cost'] == min_cost]
        
        if len(winners) > 1:
            # Randomly choose one index from the candidates (if more than one with the same cost)
            chosen_index = random.choice(winners.index)
            winner = df.loc[chosen_index, 'Population']
        else:
            winner = winners.iloc[0]['Population']  # Single winner
        # Assign the winner to winner_a for the first iteration and winner_b for the second iteration
        if a is None:
            a = winner
        else:
            b = winner

    return a, b

# SINGLE POINT CROSS

def single_point_crossover(a,b):
    '''
    Single point crossover (with fix), takes two routes (a and b)

    Parameters

    - a (list): TSP route
    - b (list): TSP route

    Returns

    child_c, child_d (list): routes with single point crossover mutation
    '''

    crossover_point = random.randint(0, len(a))
    # do the intial crossover
    child_c = a[:crossover_point] + b[crossover_point:]
    child_d = b[:crossover_point] + a[crossover_point:]
    
    # create list of missing values from the swap section. compares new child to original parent
    missing_values_c = [value for value in a[crossover_point:] if value not in child_c[crossover_point:]]
    missing_values_d = [value for value in b[crossover_point:] if value not in child_d[crossover_point:]]

    # Replace the duplicate values with the missing values in c  
    for value in child_c[crossover_point:]:
        if value in child_c[:crossover_point]:
            child_c[child_c.index(value)] = missing_values_c.pop(0)

    # Replace the duplicate values of missing values in d
    for value in child_d[crossover_point:]:
        if value in child_d[:crossover_point]:
            child_d[child_d.index(value)] = missing_values_d.pop(0)
    
    return child_c, child_d

# ORDERED CROSSOVER

def ordered_crossover(parent_a, parent_b):
    '''
    Ordered crossover for two routes

    Parameters

    - a (list): TSP route
    - b (list): TSP route

    Returns

    child_c, child_d (list): routes with ordered crossover mutation
    '''

    # Crossover points
    start, end = sorted(random.sample(range(len(parent_a)-1), 2))

    # Empty list for the crossover
    child_c = [''] * len(parent_a)
    child_d = [''] * len(parent_b)

    # Fill in the crossover
    child_c[start:end] = parent_b[start:end]
    child_d[start:end] = parent_a[start:end]

    # Values from the parents after the crossover point --> value before the crossover
    values_from_parent_a = parent_a[end:] + parent_a[:end]
    values_from_parent_b = parent_b[end:] + parent_b[:end]

    # Values missing from children after crossover
    missing_values_c = set(parent_b[start:end])
    missing_values_d = set(parent_a[start:end])

    # List without duplicates, containing the values needed to fill the child
    values_from_parent_a = [value for value in values_from_parent_a if value not in missing_values_c]
    values_from_parent_b = [value for value in values_from_parent_b if value not in missing_values_d]

    # Obtain the index of values for where to fill blank genes
    index_for_values = len(child_c[end:])

    # Complete child gene
    child_c[end:] = values_from_parent_a[:index_for_values]
    child_c[:start:] = values_from_parent_a[index_for_values:]

    child_d[end:] = values_from_parent_b[:index_for_values]
    child_d[:start:] = values_from_parent_b[index_for_values:]

    return child_c, child_d

# INVERSE MUTATION

def inversion_mutation(parent_a, parent_b):
    """
    Inversion mutation for a given route.

    Parameters:
    - parent_a (list): TSP Route
    - parent_b (list): TSP Route

    Returns:
    - child_c, child_d (list): TSP routes after inversion mutation.
    """
    # Randomly select two positions
    start, end = sorted(random.sample(range(len(parent_a)), 2))

    # Perform inversion mutation
    child_c = parent_a[:start] + parent_a[start:end+1][::-1] + parent_a[end+1:]
    child_d = parent_b[:start] + parent_b[start:end+1][::-1] + parent_b[end+1:]

    return child_c, child_d


# SWAP MUTATION

def swap_mutation(c, d, n):
    '''
    Swap mutation that swaps position of cities within routes

    Parameters

    c (list): TSP Route
    d (list): TSP Route
    n (int): number of desired swaps

    Returns e, f (list): TSP routes with swap mutation
    '''
    # create copy of c and d for mutations
    e = c
    f = d
    for _ in range(n):
        point1, point2 = random.sample(range(len(c)), 2)
        e[point1], e[point2] = e[point2], e[point1]
        f[point1], f[point2] = f[point2], f[point1]
    e = c
    f = d
    return e, f

# REPLACEMENT FUNCTIONS

def replace_weakest(df, child_1, child_2, child_1_cost, child_2_cost):
    '''
    Replace the first two weakest solutions in the population with given children.

    Parameters:
    
    - df (pd.DataFrame): DataFrame containing 'Population' and 'Cost' columns
    - child_1 (list): mutated route
    - child_2 (list): mutated route
    - child_1_cost (float): cost of mutated route
    - child_2_cost (float): cost of mutated route

    Returns:
    
    df (pd.DataFrame): DataFrame with first two weakest replaced with two children
    '''

    # Find the indices of the first two weakest routes
    weakest_indices = df.nlargest(2, 'Cost').index

    # Replace the weakest routes with the two children
    df = df.drop(weakest_indices)
    new_rows = pd.DataFrame({'Population': [child_1, child_2], 'Cost': [child_1_cost, child_2_cost]})
    df = pd.concat([df, new_rows], ignore_index=True)

    return df

def replace_first_weakest(df, child_1, child_2, child_1_cost, child_2_cost):
    '''
    Parse through list, find first two weakest solutions, replace them with e and f

    Parameters:
    
    - df (pd.DataFrame): DataFrame containing 'Population' and 'Cost' columns
    - child_1 (list): mutated route
    - child_2 (list): mutated route
    - child_1_cost (list): cost of mutated route
    - child_2_cost (list): cost of mutated route

    Population (dict): population of routes and route costs, with first two weakest replaced with two children

    '''

    new_rows = []
    # for child 1
    for index, row in df.iterrows():
        if child_1_cost < row['Cost']:
            better_child_1 = {'Population': child_1, 'Cost': child_1_cost}
            new_rows.append(better_child_1)
            # remove weaker
            df.drop(index)
            break
    # for child 2
    for index, row in df.iterrows():
        if child_2_cost < row['Cost']:
            better_child_2 = {'Population': child_2, 'Cost': child_2_cost}
            new_rows.append(better_child_2)
            # remove weaker
            df.drop(index)
            break

    # add children if possible
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        df = pd.concat([df, new_df], ignore_index=True)
    
    return df


In [100]:
# TERMINATION CRITERIA - Each generation
def genetic_algorithm(df, node_cost, max_rounds, tournament_size, crossover, replace, mutation, number_of_swaps):
    '''
    Run our genetic algorthm for max_rounds, creating and introducing them into the population

    Parameters:

    df (pd.Dataframe): Dataframe containing 'population' and 'cost' columns. It is the information for the starting population
    node_cost(dict): Cities as key, list of all other cities to travel to as value
    max_rounds (int): Maximum amount of generations we want. How many times we want to run the GA
    tournament_size(int): How many routes we want to include in the tournament
    crosspver (function): The type of crossover we want to include, name of function
    replace (funciton): The type of replacement strategy we want to include, name of function

    Returns:

    results (pd.Dataframe): Dataframe that contains the minimum cost for a duartion of time and the associated route
    
    '''
    rounds = 0
    data = []
    while rounds < max_rounds:
        # Tournament winners, using initial population and adjustable size
        a, b = tournament(df, tournament_size)
        # Crossover, using the winners
        c, d = crossover(a,b)
        # Swap using the children, number of swaps
        if mutation == swap_mutation:
            e, f = mutation(c, d, n = number_of_swaps)
        else:
            e, f = mutation(c, d)

        # Calculate the cost of the children routes
        e_cost, f_cost = calculate_route_cost(e, node_cost),calculate_route_cost(f, node_cost)
        
        # Replace the children back into the population using either:
        #population = replace_first_weakest(cost_to_route, e, f, e_cost, f_cost)
        df = replace(df, e, f, e_cost, f_cost)
        min_cost_row = df.loc[df['Cost'].idxmin()]
        data.append({'Population': min_cost_row['Population'], 'Cost': min_cost_row['Cost'], 'Generation': rounds})   
        rounds +=1
    results = pd.DataFrame(data)
    #results = results.sort_values('Cost')
    return results

In [97]:
# TERMINATION CRITERIA - New best
def genetic_algorithm(df, node_cost, max_rounds, tournament_size, crossover, replace, mutation, number_of_swaps):
    '''
    Run our genetic algorthm for max_rounds, creating and introducing them into the population

    Parameters:

    df (pd.Dataframe): Dataframe containing 'population' and 'cost' columns. It is the information for the starting population
    node_cost(dict): Cities as key, list of all other cities to travel to as value
    max_rounds (int): Maximum amount of generations we want. How many times we want to run the GA
    tournament_size(int): How many routes we want to include in the tournament
    crosspver (function): The type of crossover we want to include, name of function
    replace (funciton): The type of replacement strategy we want to include, name of function

    Returns:

    results (pd.Dataframe): Dataframe that contains the minimum cost for a duartion of time and the associated route
    
    '''
    rounds = 0
    data = []
    current_best = df.loc[df['Cost'].idxmin()]
    data.append({'Population': current_best['Population'], 'Cost': current_best['Cost'], 'Generation':rounds})
    while rounds < max_rounds:
        # Tournament winners, using initial population and adjustable size
        a, b = tournament(df, tournament_size)
        # Crossover, using the winners
        c, d = crossover(a,b)
        # Swap using the children, number of swaps
        
        if mutation == swap_mutation:
            e, f = mutation(c, d, number_of_swaps)
        else:
            e, f = mutation(c, d)

        # Calculate the cost of the children routes
        e_cost, f_cost = calculate_route_cost(e, node_cost),calculate_route_cost(f, node_cost)
        
        # Replace the children back into the population using either:
        #population = replace_first_weakest(cost_to_route, e, f, e_cost, f_cost)
        df = replace(df, e, f, e_cost, f_cost)
        min_cost_row = df.loc[df['Cost'].idxmin()]
        if min_cost_row['Cost'] < current_best['Cost']:
            data.append({'Population': min_cost_row['Population'], 'Cost': min_cost_row['Cost'], 'Generation':rounds}) 
            current_best = min_cost_row  
        rounds +=1
    results = pd.DataFrame(data)
    results = results.sort_values('Cost').reset_index(drop=True) 
    return results

In [107]:
# CONTROLS FOR EA 

#  Generate initial population
xml_file = 'burma14.xml'  # Replace with the actual XML file path
population_size = 100
seed_value = 1
df, node_cost = initialize_population(xml_file, population_size, seed_value)

# Controls for evolution (first item in list used for default)
max_rounds = 50
tournament_size= [2, 5, 10, 20] # List if testing different tournament sizes
mutations = [swap_mutation, inversion_mutation]  # List if testing different mutation strategies
number_of_swaps = [0, 2, 5]  # List if testing different number of swaps
crossovers = [single_point_crossover, ordered_crossover]   # List if testing different crossover strategies
replacements = [replace_weakest, replace_first_weakest] # List if testing different replacement strategies

# Save file - change as needed
save_file = 'burma_replace_results.csv'
save_plot = 'burma_replace_plot.png'

In [ ]:
# Tournament size test

country_results = pd.DataFrame(columns=['Tournament size', 'Min cost', 'Route'])
for _ in range(1,11):
    data = []
    for n in tournament_size:
    # Assuming `genetic_algorithm` returns a DataFrame named `df` with results
        results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=n, number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[0])
        entry = pd.DataFrame.from_dict({ 
                                        'Tournament size': [n], 
                                        'Min cost': [results['Cost'].iloc[0]], 
                                        'Route': [results['Population'].iloc[0]],
                                        'Generation': [results['Generation'].iloc[0]]
                                        })
        
        data.append(entry)
    country_results = pd.concat([country_results, pd.concat(data)], ignore_index = True)
        

country_results.to_csv(save_file, index=False)

    

In [ ]:
# Swap Mutation test

country_results = pd.DataFrame(columns=['Swap Size', 'Min cost', 'Route', 'Generation'])
for _ in range(1,11):
    data = []

    # Inverse mutation
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[1])
    entry = pd.DataFrame.from_dict({ 
                                    'Swap Size': ['Inverse'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                    })
    data.append(entry)
    # Simple swap
    for n in number_of_swaps:
    # Iterate through number of swaps
        results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[0])
        entry = pd.DataFrame.from_dict({ 
                                        'Swap Size': [n], 
                                        'Min cost': [results['Cost'].iloc[0]], 
                                        'Route': [results['Population'].iloc[0]],
                                        'Generation': [results['Generation'].iloc[0]]
                                        })


        data.append(entry)
    country_results = pd.concat([country_results, pd.concat(data)], ignore_index = True)
        

country_results.to_csv(save_file, index=False)

In [ ]:
# Crossover test

country_results = pd.DataFrame(columns=['Crossover', 'Min cost', 'Route', 'Generation'])
for _ in range(1,11):
    data = []
# Single point crossover
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[0])
    entry = pd.DataFrame.from_dict({ 
                                    'Crossover': ['Single Point Crossover'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })
    data.append(entry)
# Ordered crossover
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[1], replace = replacements[0], mutation = mutations[0])
    entry = pd.DataFrame.from_dict({ 
                                    'Crossover': ['Ordered Crossover'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })

    data.append(entry)

    country_results = pd.concat([country_results, pd.concat(data)], ignore_index = True)
        

country_results.to_csv(save_file, index=False)

In [ ]:
# Replacement test

country_results = pd.DataFrame(columns=['Replacement', 'Min cost', 'Route', 'Generation'])
for _ in range(1,11):
    data = []
# Replace weakest
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[0])
    entry = pd.DataFrame.from_dict({ 
                                    'Replacement': ['Replace Weakest'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })
    data.append(entry)
# Replace first weakest
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[1], mutation = mutations[0])
    entry = pd.DataFrame.from_dict({ 
                                    'Replacement': ['Replace First Weakest'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })

    data.append(entry)

    country_results = pd.concat([country_results, pd.concat(data)], ignore_index = True)
        

country_results.to_csv(save_file, index=False)

In [ ]:
# Individual Plots

import matplotlib.pyplot as plt
import seaborn as sns


results = pd.read_csv(save_file)
# Assuming burma_results has been populated as desired
# Modify the column names if needed
x = 'Generation'
y = 'Min cost'
hue = results.columns.values[0]  # Grab adjusted variable

# Set seaborn style for improved aesthetics (optional)
sns.set(style="whitegrid")

# Define a vibrant color palette
color_palette = sns.color_palette("husl", n_colors=len(results[hue].unique()))

plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

# Use Seaborn scatterplot with 'hue' for different colors
sns.scatterplot(data=results, x=x, y=y, hue=hue, palette=color_palette, marker='o', s=100)  # Adjust 's' for marker size

# Customize the plot
plt.title('Scatter Plot of Minimum Cost for Brazil58')
plt.xlabel(x)
plt.ylabel(y)
plt.legend(title=hue, loc='upper right')  # Adjust the legend location as needed


# Save the plot as an image (optional)
plt.savefig(save_plot, bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Joint plots

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Assuming 'burma_results.csv' and 'brazil_results.csv' contain the respective data
burma_results = pd.read_csv('burma_replace_results.csv')
brazil_results = pd.read_csv('brazil_replace_results.csv')

# Modify the column names if needed
x = 'Generation'
y_burma = 'Min cost'  # Rename y-axis for Burma
y_brazil = 'Min cost'  # Rename y-axis for Brazil
hue = results.columns.values[0]  # Grab adjusted variable

# Set seaborn style for improved aesthetics (optional)
sns.set(style="whitegrid")
color_palette = sns.color_palette("husl", n_colors=max(len(burma_results[hue].unique()), len(brazil_results[hue].unique())))

# Create subplots with shared x-axis and different y-axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=False)

# Plot for Burma
sns.scatterplot(data=burma_results, x=x, y=y_burma, hue=hue, palette=color_palette, marker='o', s=100, ax=ax1)
ax1.set_title('Burma14')
ax1.set_xlabel(x)
ax1.set_ylabel(y_burma)
ax1.legend(title=hue, loc='upper right')

# Plot for Brazil
sns.scatterplot(data=brazil_results, x=x, y=y_brazil, hue=hue, palette=color_palette, marker='o', s=100, ax=ax2)
ax2.set_title('Brazil58')
ax2.set_xlabel(x)
ax2.set_ylabel(y_brazil)
ax2.legend(title=hue, loc='upper right')

# Adjust layout
plt.tight_layout()

# Save the plot
plt.savefig(save_plot, bbox_inches='tight')

# Show the plots
plt.show()

In [927]:
# Generate data for the different EA's
xml_file = 'brazil58.xml'
max_rounds = 5000
population_size = 100
seed_value=1


'''
# EA1 - maximum diversiy
tournament_size= 2
crossover = ordered_crossover
replace = replace_first_weakest
swap = inverse

# EA2 - minimal diversity

tournament_size = 20
crossover = single_point_crossover
replace = replace_weakest
number_of_swaps = 0

# EA3

tournament_size = 10
crossover = ordered_crossover
replace = replace_weakest
swap = inverse 

# EA4

tournament_size = 10
crossover = single_point_crossover
replace = replace_first_weakest
swap = inverse 
'''

# Save file - change as needed
save_file = 'brazil_bestea_results.csv'
save_plot = 'brazil_bestea_plot.png'

country_results = pd.DataFrame(columns=['EA', 'Min cost', 'Route', 'Generation'])
for seed in range(1,11):
    data = []
# Generate initial population
    df, node_cost = initialize_population(xml_file, population_size, seed)
    
# EA1
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[0], number_of_swaps=number_of_swaps[0], crossover = crossovers[1], replace = replacements[1], mutation = mutations[1])
    entry = pd.DataFrame.from_dict({ 
                                    'EA': ['EA1'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })
    data.append(entry)
# EA2
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[3], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[0], mutation = mutations[0])
    entry = pd.DataFrame.from_dict({ 
                                    'EA': ['EA2'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })       
    data.append(entry)
# EA3
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[2], number_of_swaps=number_of_swaps[0], crossover = crossovers[1], replace = replacements[0], mutation = mutations[1])
    entry = pd.DataFrame.from_dict({ 
                                    'EA': ['EA3'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })       
    data.append(entry)
# EA4
    results = genetic_algorithm(df, node_cost, max_rounds, tournament_size=tournament_size[2], number_of_swaps=number_of_swaps[0], crossover = crossovers[0], replace = replacements[1], mutation = mutations[1])
    entry = pd.DataFrame.from_dict({ 
                                    'EA': ['EA4'], 
                                    'Min cost': [results['Cost'].iloc[0]], 
                                    'Route': [results['Population'].iloc[0]],
                                    'Generation': [results['Generation'].iloc[0]]
                                       })       
    data.append(entry)
    country_results = pd.concat([country_results, pd.concat(data)], ignore_index = True)
        

country_results.to_csv(save_file, index=False)
    


In [108]:
# Best EA - plot to evaluate perfromance of diff EA's
import seaborn as sns
import matplotlib.pyplot as plt

burma_data = pd.read_csv(save_file)
brazil_data = pd.read_csv(save_file)

# Create subplots with shared x-axis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=False)

# Plot for burma_data
sns.boxplot(x='EA', y='Min cost', data=burma_data, ax=ax1)
ax1.set_xlabel('Evolutionary Algorithms')
ax1.set_ylabel('Minimum Cost')
ax1.set_title('Burma Data')

# Plot for brazil_data
sns.boxplot(x='EA', y='Min cost', data=brazil_data, ax=ax2)
ax2.set_xlabel('Evolutionary Algorithms')
ax2.set_ylabel('Minimum Cost')
ax2.set_title('Brazil Data')

# Adjust layout and show plots
plt.tight_layout()

plt.savefig(save_plot, bbox_inches='tight')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'burma_bestea_results.csv'